In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
#import libraries
import time
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
import string

#for training
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
#for word
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import cross_val_score
from keras.utils import to_categorical

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
url= 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
dataset = pd.read_csv(url, sep='\t')

In [55]:
dataset.shape

(156060, 4)

In [56]:
dataset.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [57]:
print(dataset.isnull().values.any())

False


In [58]:
dataset['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['Phrase'],dataset['Sentiment'], test_size=0.3, random_state=2003)

In [60]:
X_train.dropna
X_test.dropna
print(X_train.shape)
print(X_test.shape)

(109242,)
(46818,)


In [0]:
#porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome
remove_stopwords = True
useStemming = False
useLemma = True
removePuncs = True

In [0]:
Preprocess_train_data=[]
for x in range(0,len(X_train.values)):
    tmpReview=[]
    for w in nltk.word_tokenize(X_train.values[x]):
        newWord = str(w).lower() #Set newWork to be the updated word
        if remove_stopwords and (w in stopwords_en):#if the word is a stopword & we want to remove stopwords
            continue #skip the word and don’t had it to the normalized review
        if removePuncs and (w in punctuations):#if the word is a punc. & we want to remove punctuations
            continue #skip the word and don’t had it to the normalized review
        if useStemming: #if useStemming is set to True
            #Keep one stemmer commented out
            #newWord = porter.stem(newWord) #User porter stemmer
            newWord = lancaster.stem(newWord) #Use Lancaster stemmer
        if useLemma:
            newWord = wordnet_lemmatizer.lemmatize(newWord)
        tmpReview.append(newWord) #Add normalized word to the tmp review
    Preprocess_train_data.append(' '.join(tmpReview))

In [0]:
Preprocess_test_data=[]
for x in range(0,len(X_test.values)):
    tmp_test_Review=[]
    for w in nltk.word_tokenize(X_test.values[x]):
        newWord = str(w).lower() #Set newWork to be the updated word
        if remove_stopwords and (w in stopwords_en):#if the word is a stopword & we want to remove stopwords
            continue #skip the word and don’t had it to the normalized review
        if removePuncs and (w in punctuations):#if the word is a punc. & we want to remove punctuations
            continue #skip the word and don’t had it to the normalized review
        if useStemming: #if useStemming is set to True
            #Keep one stemmer commented out
            #newWord = porter.stem(newWord) #User porter stemmer
            newWord = lancaster.stem(newWord) #Use Lancaster stemmer
        if useLemma:
            newWord = wordnet_lemmatizer.lemmatize(newWord)
        tmp_test_Review.append(newWord) #Add normalized word to the tmp review
    Preprocess_test_data.append(' '.join(tmp_test_Review))

In [64]:
print(len(Preprocess_train_data))
len(Preprocess_test_data)

109242


46818

In [65]:
dataset.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [66]:
Y_train.value_counts()

2    55595
3    23055
1    19203
4     6468
0     4921
Name: Sentiment, dtype: int64

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

# Transform each text into a vector of word counts
#vectorizer = CountVectorizer(stop_words="english",ngram_range=(1, 1))
#vectorizer_tfid = TfidfVectorizer(stop_words="english",ngram_range=(1, 1))
vectorizer_tfid = TfidfVectorizer(ngram_range=(1, 2),max_features=2000)

#X = vectorizer_tfid.fit_transform(Preprocess_train_data)
Y = dataset['Sentiment']
Train_series = pd.Series(Preprocess_train_data)
Test_series = pd.Series(Preprocess_test_data) 
x_train = vectorizer_tfid.fit_transform(Train_series)
y_train = Y_train
x_test = vectorizer_tfid.transform(Test_series)
y_test = Y_test

In [68]:
x_train.shape,x_test.shape

((109242, 2000), (46818, 2000))

In [69]:
x_train

<109242x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 307234 stored elements in Compressed Sparse Row format>

In [70]:
x_train_np = x_train.toarray()
y_train_np = np.array(y_train)
# Convert the testing data
x_test_np = x_test.toarray()
y_test_np = np.array(y_test)
print(x_train_np.shape)
print(y_train_np.shape)

(109242, 2000)
(109242,)


In [71]:
x_train_np=x_train_np.reshape(x_train_np.shape[0],x_train_np.shape[1],1)
x_test_np=x_test_np.reshape(x_test_np.shape[0],x_test_np.shape[1],1)
print(x_train_np.shape)
print(x_test_np.shape)

y_train_np = to_categorical(y_train_np)
print(y_train_np.shape)
y_test_np = to_categorical(y_test_np)
print(y_test_np.shape)


(109242, 2000, 1)
(46818, 2000, 1)
(109242, 5)
(46818, 5)


In [0]:
#bow_feature= pd.DataFrame(x_train_np, columns=vectorizer.get_feature_names())
#bow_feature.head(30)

In [0]:
from keras import backend as K
def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0 ,1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0 ,1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from keras.layers import Activation, GlobalMaxPooling1D, AvgPool1D
from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import adam

In [75]:
model = Sequential()
#model.add(Dense(64,input_shape=(fea_matrix.shape[1],fea_matrix.shape[2])))
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(x_train_np.shape[1],x_train_np.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=512, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
#model.add(Dense(50, input_dim=2, activation='relu'))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))
#model.add(Activation('softplus'))
model.summary()  
#opt=adam(lr=0.0001, decay= 0.01)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 1999, 64)          192       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 999, 64)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 998, 128)          16512     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 499, 128)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 498, 256)          65792     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 249, 256)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 248, 512)         

In [0]:
history2= model.fit(x_train_np, y_train_np, epochs=50, verbose=1, validation_split=0.2, batch_size = 128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 87393 samples, validate on 21849 samples
Epoch 1/50





 3456/87393 [>.............................] - ETA: 29:34 - loss: 1.3523 - acc: 0.4942 - f1_m: 0.2260 - precision_m: 0.4195 - recall_m: 0.1788

In [0]:
#testing results
print(model.metrics_names)
model.evaluate(x_test_np, y_test_np)

In [0]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
# Plot training & validation accuracy values
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.show()
plt.savefig('/content/drive/My Drive/NLP_prac/Subplot3.png', format='png', dpi=1200)
